In [171]:
import re 
import json 
from typing import List , Union , Dict , Set ,Tuple, Optional
from graphviz import Digraph
import itertools 

In [172]:
'''
Helper Functions 
 
'''
def split_input(input: str):
    split_list = []
    opening_match_found = False 
    match = ""
    for char in input: 
        if char == '[' :
            opening_match_found = True 
        elif char == ']': 
            opening_match_found = False 
            split_list.append('['+ match +']')
            match = "" 
        elif  opening_match_found == 0 :
            split_list.append (char) 
        else : 
            match =  match + char 
    return split_list

def visualize_nfa(nfa_json):
    # Create a new directed graph
    dot = Digraph(format='png')
    
    # Set global graph attributes for visual clarity
    dot.attr(rankdir='LR')  # Left to right layout

    # Add states to the graph
    start_state = nfa_json.get("startingState")
    for state_name, state_info in nfa_json.items():
        if state_name == "startingState":
            continue

        # Determine if this state is an accepting state
        is_accepting = state_info["isTerminatingState"]

        # Customize node style based on state type
        if state_name == start_state:
            dot.node(state_name, shape='circle', color='green', label=state_name)  # Start state
        elif is_accepting:
            dot.node(state_name, shape='doublecircle', color='blue', label=state_name)  # Accepting state
        else:
            dot.node(state_name, shape='circle', label=state_name)

    # Add transitions
    for state_name, state_info in nfa_json.items():
        if state_name == "startingState":
            continue

        # Iterate over transitions and add them as edges
        for input_symbol, destinations in state_info.items():
            if input_symbol == "isTerminatingState":
                continue
            
            if isinstance(destinations, list):
                # If there are multiple destinations, create separate edges for each
                for destination in destinations:
                    label = "ε" if input_symbol == "~" else input_symbol  # Represent epsilon with 'ε'
                    dot.edge(state_name, destination, label=label)
            else:
                # Single destination, normal case
                label = "ε" if input_symbol == "~" else input_symbol  # Represent epsilon with 'ε'
                dot.edge(state_name, destinations, label=label)

    # Render and view the graph
    dot.render('nfa_graph', view=True)



In [173]:


'''
This preporcessing is applied to the regex before applying shunting yard algorithm in order to : 
1) reduce all the regex to these operations only ( * , | , parenthes , concat )
2) add a concatination symbol between characters to be recognized by the algorithm as an operator 
3) replace every [match] with one char to be treated as other alphanumeric 
'''  
def preprocessing(input : str) :
# input= 'a?a((cd)|(a|b))b+bb' 
    #Step1: Replace zero or one symbol '?'
    step_1 = re.sub(r'(\w)\?', r'(\1|~)', input)
    #Step2: Replace one or more symbol '+'
    step_2 = re.sub(r'(\w)\+', r'\1\1*', step_1)
    #Step3: Add concat symbol before every [ or (  if they are not at the start of the regex and they are not preceded by ?
    pattern_before = re.compile(r'''
        (?<!^)      # Negative lookbehind assertion to ensure the position is not at the start of the string
        (?<!\?)     # Negative lookbehind assertion to ensure the position is not preceded by '?'
        (?=[\[\(])  # Positive lookahead assertion to match '[' or '(' without consuming them
    ''', re.VERBOSE)
    step_3 = pattern_before.sub('?', step_2) 
    #Step 4: Add concat symbol after every ] or ) if they are not the end of regex OR they are not followed by * and not followed by ? 
    pattern_after = re.compile(r'''
        (?<=[\]\)])  # Positive lookbehind assertion to match ']' or ')' without consuming them
        (?!$)        # Negative lookahead assertion to ensure the position is not at the end of the string
        (?![\*\?])   # Negative lookahead assertion to ensure the position is not followed by '*' or '?'
    ''', re.VERBOSE)
    step_4 = pattern_after.sub('?', step_3)
    #Step 5 : Add concat after every * if its not the end of regex and its not follwed by star 
    pattern_star = re.compile(r'''
        \*          # Match the '*' character
        (?!$)       # Negative lookahead assertion to ensure the position is not at the end of the string
        (?![\?])    # Negative lookahead assertion to ensure the position is not followed by '?'                      
    ''', re.VERBOSE)
    step_5 = pattern_star.sub('*?', step_4)
    #Step 6 : Add concat after every alphanum or dot if its followed by alphanumeric or dot 
    pattern_alnum_dot = re.compile(r'''
        ([a-zA-Z0-9\.])  # Match any alphanumeric character or dot
        (?=[a-zA-Z0-9\.])  # Positive lookahead assertion to ensure it is followed by another alphanumeric character or dot
    ''', re.VERBOSE)
    step_6 = pattern_alnum_dot.sub(r'\1?', step_5)
    return split_input(step_6) 




In [174]:

'''
Postfix notation removes the need for parentheses and allows computer programs to read in 
mathematical expressions one symbol after the other, instead of worrying about operator precedence 
and parentheses during computation. 

'''
def shuntingYard(input) :
    precedence_dict = {'*': 3, '?': 2, '|': 1}
    out =[]
    operator_stack = []
    for char in input :
        # If the input is alphanumeric then append to the output regex 
        if char.isalnum() or char == '.' or len(char) > 1 :
            out.append(char)
        # If the input is an operator
        elif  char in precedence_dict.keys() :
            # The first operator in the stack 
            if len(operator_stack) == 0:
                operator_stack.append(char) 
            #  Any operator shouldn't be compared to an opening parenthes , if an opening parenthes is on the top of the stack Just add the char to the stack directly 
            elif operator_stack[-1] =='('or precedence_dict[ operator_stack[-1] ] < precedence_dict[char] :
                operator_stack.append(char)
            # If the operator on the top of the stack is the same as the current char then pop one to the output regex and leave the other in tha stack 
            # If two consecutive opening parenthes comes we need them both to be in the stack and not popped to the output because they will be deleted later 
            elif operator_stack[-1] != '(' and precedence_dict[ operator_stack[-1] ] == precedence_dict[char] : 
                out.append(char) 
            #If the operator at the top of the stack has higher precedence that the current operator then pop to the output until we can push the current operator to the stack 
            else : 
                while (len(operator_stack)>0 and operator_stack[-1] != '('and precedence_dict[ operator_stack[-1] ]  >= precedence_dict[char]) :
                    popped_operator = operator_stack.pop()
                    out.append(popped_operator)
                operator_stack.append(char) 
        elif char == '(' :
            operator_stack.append(char)
        # The current char is closing parenthes -> pop from the operator stack to the output until you reach an opening parenthes
        elif char == ')' :
            while operator_stack:
                operator = operator_stack.pop()
                if operator == '(':
                    break
                out.append(operator)
        # print ("parsing char :",char,"output:",out,"stack",operator_stack)
        # print("************************")
    out.extend(operator_stack[::-1]) 
    # print ("Final out :",out)
    return out 

    # print("Final stack :",operator_stack)


In [175]:
'''
Datastructures defined
 
'''
class Transition:
    def __init__(self, destination , input):
        self.destination = destination
        self.input = input
    def __repr__(self) -> str:
        return f"Transition(destination={self.destination.state_name}, input='{self.input}')"


class State:
    state_counter = 0  

    def __init__(self):
        self.state_name = 'S'+ str(State.state_counter) 
        State.state_counter += 1
        self.transitions = [] 
        self.isTerminatingState = False 

    def add_transition(self, destination, input):
        self.transitions.append(Transition(destination, input))

    def findTransitionByInput(self , input : str) :
        transition : Transition 
        for transition in self.transitions : 
            if transition.input == input :
                return True , transition.destination 
        return None , None 

    def to_dict(self):
        state_dict = {
            "isTerminatingState": self.isTerminatingState
        }
        for transition in self.transitions:
            if transition.input in state_dict:
                # If key already exists, append to the list of destinations
                if isinstance(state_dict[transition.input], list):
                    state_dict[transition.input].append(transition.destination.state_name)
                else:
                    # If it's not already a list, make it a list
                    state_dict[transition.input] = [state_dict[transition.input], transition.destination.state_name]
            else:
                # If key doesn't exist, add it
                state_dict[transition.input] = transition.destination.state_name
        return state_dict

    def __repr__(self):
        return json.dumps(self.to_dict(), indent=2)
    
class NFA:
    def __init__(self):
        self.states = []
        self.start_state:State = None 
        #TODO : is this list or one element 
        self.accept_states : List [State] = []

    def add_states(self, states: Union[List[State], List[List[State]]]):
        # Flatten the list of lists if necessary
        def flatten(states_list: List[Union[State, List[State]]]) -> List[State]:
            flat_list = []
            for state in states_list:
                if isinstance(state, list):
                    flat_list.extend(flatten(state))
                else:
                    flat_list.append(state)
            return flat_list

        states = flatten(states)
        # Ensure every state.isTerminatingState = False
        for state in states:
            state.isTerminatingState = False
        
        self.states.extend(states)
        if not self.start_state and states:
            self.start_state = states[0]
        if states:
            self.accept_states.append(states[-1])
            states[-1].isTerminatingState = True

        
    def add_state(self, state , is_start = False , is_accept = False  ):
        self.states.append(state)
        if is_start:
            self.start_state = state
        if is_accept:
            self.accept_states.append(state)
        return state
    
    def to_dict(self):
        nfa_dict = {
            "startingState": self.start_state.state_name if self.start_state else None
        }
        for state in self.states:
            nfa_dict[state.state_name] = state.to_dict()
        return nfa_dict

    def __repr__(self):
        return json.dumps(self.to_dict(), indent=2)

In [176]:
'''
Finally : Thomsons construction Algorithm 
'''

def create_nfa_initial_states() : 
    start_state = State()
    end_state = State()
    end_state.isTerminatingState = True 
    return start_state , end_state
    
def alphanumeric_nfa(char) : 
    start_state , end_state = create_nfa_initial_states()
    start_state.add_transition(end_state , char)
    nfa = NFA() 
    nfa.add_states([start_state , end_state])
    return nfa 

def zero_or_more_nfa(operand: NFA) : 
    new_start_state , new_end_state = create_nfa_initial_states()
    new_start_state.add_transition(operand.start_state , '~')
    new_start_state.add_transition(new_end_state ,'~')
    operand.accept_states[0].add_transition(operand.start_state ,'~')
    operand.accept_states[0].add_transition(new_end_state,'~')
    nfa = NFA ()
    nfa.add_states([new_start_state , operand.states , new_end_state])
    return nfa

def union_nfa(operand1: NFA , operand2: NFA) :
    new_start_state , new_end_state = create_nfa_initial_states()
    new_start_state.add_transition(operand1.start_state , '~')
    new_start_state.add_transition(operand2.start_state , '~')
    operand1.accept_states[0].add_transition(new_end_state , '~')
    operand2.accept_states[0].add_transition(new_end_state , '~')
    nfa = NFA()
    nfa.add_states([new_start_state , operand1.states , operand2.states , new_end_state])
    return nfa

def concatinate_nfa(operand1: NFA , operand2: NFA) :
    operand1.accept_states[0].add_transition(operand2.start_state , '~')
    nfa = NFA()
    nfa.add_states([operand1.states , operand2.states])
    return nfa

def constructNFA(input ) : 
    stack_NFA = []
    for char in input : 
        if char.isalnum() or char == '~' : 
            stack_NFA.append(alphanumeric_nfa(char))  
        elif char == '*':
            stack_NFA.append(zero_or_more_nfa(stack_NFA.pop()))
        elif char == '|': 
            operand2 = stack_NFA.pop() 
            operand1 = stack_NFA.pop()
            stack_NFA.append(union_nfa(operand1, operand2))
        elif char == '?' :
            operand2 = stack_NFA.pop() 
            operand1 = stack_NFA.pop()
            stack_NFA.append(concatinate_nfa(operand1, operand2))

    return stack_NFA[0] 

def ThomsonsConstruction (input : str) -> NFA : 
    preprocessed = preprocessing(input) 
    shunting_yard = shuntingYard(preprocessed)
    return constructNFA(shunting_yard)  
nfa = ThomsonsConstruction('0|1(0|1)*00')
nfa_json = nfa.to_dict()
# Print the JSON-formatted string
#print(json.dumps(nfa_json, indent=2))
visualize_nfa(nfa_json)
   

In [177]:
'''
NFA to DFA 

'''
class DFAState:
    def __init__(self, nfa_states: Set[State]):
        self.nfa_states = nfa_states
        self.transitions: Dict[str, 'DFAState'] = {}
        self.isTerminatingState = any(state.isTerminatingState for state in nfa_states)
        self.state_name = ','.join(sorted(state.state_name for state in nfa_states))

    def add_transition(self, input: str, destination: 'DFAState'):
        self.transitions[input] = destination
    
    def get_transitions_inputs(self) :
        return tuple(sorted(self.transitions.keys()))
    
    def to_dict(self):
        state_dict = {
            "isTerminatingState": self.isTerminatingState
        }
        for input, destination in self.transitions.items():
            state_dict[input] = destination.state_name
        return state_dict

    def __repr__(self):
        return json.dumps(self.to_dict(), indent=2)
    
class DFA:
    def __init__(self):
        self.states: List[DFAState] = []
        self.start_state: DFAState = None

    def add_state(self, state: DFAState, is_start=False):
        self.states.append(state)
        if is_start:
            self.start_state = state
    def find_DFA_state_by_NFA_states(self, nfa_states: Set[State]) -> Optional[DFAState]:
        for dfa_state in self.states:
            if dfa_state.nfa_states == nfa_states:
                return dfa_state
        return None
    def to_dict(self):
        dfa_dict = {
            "startingState": self.start_state.state_name if self.start_state else False
        }
        for state in self.states:
            dfa_dict[state.state_name] = state.to_dict()
        return dfa_dict

    def __repr__(self):
        return json.dumps(self.to_dict(), indent=2)
# nfa = ThomsonsConstruction('a(c|d)b*')

def epsilonClosure(state : State) -> Set[State] :
    states : Set[State] = {state} 
    transition : Transition 

   
    for transition in state.transitions : 
        if transition.input == '~':
            states.add(transition.destination)
            states.update(epsilonClosure(transition.destination))

    return states 


def subsetConstruction(nfa : NFA) -> NFA :
    
    dfa = DFA()
    start_dfa_state = DFAState(epsilonClosure(nfa.start_state)  ) 
    dfa.add_state(start_dfa_state , is_start = True)
    work_list = [start_dfa_state] 
    while (work_list):
        transition : Transition
        transitions_dict: Dict[str, List[State]] = {} 
        current_dfa_state = work_list.pop() 
        #  if current_dfa_state in processed_states:
        #     continue
        # processed_states.add(current_dfa_state)


        for state in current_dfa_state.nfa_states :
            for transition in state.transitions : 
                if transition.input != '~':
                    if transition.input not in transitions_dict.keys() :
                        transitions_dict[transition.input] = [transition.destination]
                    else : 
                        transitions_dict[transition.input].append(transition.destination)
       
       # print("dictionary of transitions :",transitions_dict)
        for key , value in transitions_dict.items() : 
            epsilon_closures : Set[State] = set()

            for nfa_state in value : 
                epsilon_closures.update(epsilonClosure(nfa_state))

            existing_dfa_state = dfa.find_DFA_state_by_NFA_states(epsilon_closures)
            if existing_dfa_state is None:
                new_dfa_state = DFAState(epsilon_closures)
                dfa.add_state(new_dfa_state)
                work_list.append(new_dfa_state)
            else:
                new_dfa_state = existing_dfa_state

            current_dfa_state.add_transition(key, new_dfa_state)
    return dfa 
dfa = subsetConstruction(nfa) 
#print (dfa)

def renameDFA(dfa : DFA)-> DFA :
    counter = 1 #TODO : LET THIS COUNTER =0 ANA 3MLTO KDA BS 3SHAN BA TEST 
    for state in dfa.states : 
        state.state_name = 'S'+ str(counter) 
        counter += 1
    return dfa
print("hello",(dfa.states[0].transitions))
print("**************")
print(renameDFA(dfa)) 




hello {'1': {
  "isTerminatingState": false,
  "0": "S11,S12,S13,S14,S4,S5,S6,S8,S9",
  "1": "S11,S12,S4,S6,S7,S8,S9"
}, '0': {
  "isTerminatingState": true
}}
**************
{
  "startingState": "S1",
  "S1": {
    "isTerminatingState": false,
    "1": "S2",
    "0": "S3"
  },
  "S2": {
    "isTerminatingState": false,
    "0": "S4",
    "1": "S5"
  },
  "S3": {
    "isTerminatingState": true
  },
  "S4": {
    "isTerminatingState": false,
    "0": "S6",
    "1": "S5"
  },
  "S5": {
    "isTerminatingState": false,
    "0": "S4",
    "1": "S5"
  },
  "S6": {
    "isTerminatingState": true,
    "0": "S6",
    "1": "S5"
  }
}


In [ ]:
''' 
DFA Minimization  without comments
''' 
def get_group_index(state: DFAState, groups: List[Set[DFAState]]) -> int:
        for i, group in enumerate(groups):
            if state in group:
                return i
            
def create_transition_dict(subgroup: Set[DFAState],  groups: List[Set[DFAState]]) -> Dict[Tuple[int, ...], Set[DFAState]]:

    transition_dict: Dict[Tuple[int, ...], Set[DFAState]] = {}
    input_symbols = list(subgroup)[0].get_transitions_inputs() 
    
    for state in subgroup :
        transition_key_elements = []
        for input_symbol in input_symbols:
            destination_state = state.transitions[input_symbol]
            # Get the index of the group to which the destination state belongs
            group_index = get_group_index(destination_state, groups)
            transition_key_elements.append(group_index)
            transition_key = tuple(transition_key_elements)
        # Group states by their transition pattern
        if transition_key in transition_dict:
            transition_dict[transition_key].add(state)
        else:
            transition_dict[transition_key] = {state}
    return transition_dict

def partition_groups_based_on_transitions(groups : List[Set[DFAState]]) -> List[Set[DFAState]]:
    new_groups : List[Set[DFAState]] =  [] 
    print("total groups before partinioning :" ,len(groups))
    for i, group in enumerate(groups) : 
        transitions_dict: Dict[Tuple[str, ...], Set[str]] = {}
        for state in group:
            transitions = state.get_transitions_inputs()
            if transitions in transitions_dict:
                transitions_dict[transitions].add(state)
            else:
                transitions_dict[transitions] = {state}
       
        for value in transitions_dict.values() :
            new_groups.append(value) 
    #         print ("**********************iteration"+str(i)+"************************")
    #         print(len(new_groups))
    # print ("old groups length :",len(groups))
    # print ("new groups length",len(new_groups)) 
    # print("dict:\n",transitions_dict)
    # # for i,group in enumerate(new_groups) :
    # #     print ("**********************g"+str(i)+"************************")
    # #     print(group)
    return new_groups

def minimizeDFA(dfa : DFA) -> DFA: 
    # PASS 1 
    accepting_states : Set[DFAState] = set()
    none_accepting_states : Set[DFAState] = set()
    for state in dfa.states : 
        if state.isTerminatingState : 
            accepting_states.add(state)
        else :
            none_accepting_states.add(state)
    
    groups : List[Set[DFAState]] = [accepting_states ,  none_accepting_states] 
    groups = partition_groups_based_on_transitions(groups)
   
    #^ PASS TWO : 
    while True:
        new_groups: List[Set[DFAState]] = []
        for i, group in enumerate(groups): 
            if(len(group) == 1):
                new_groups.append(group)
                continue 
            if(list(group)[0].get_transitions_inputs() == ()): # no transitions
                new_groups.append(group)
                continue
            transition_dict = create_transition_dict(group, groups)
            for  states in transition_dict.values():
                new_groups.append(states)
        if new_groups == groups:
            print("Groups have stabilized, exiting.")
            break
        groups = new_groups
    return groups
#print (minimizeDFA(dfa))
print(len(minimizeDFA(dfa)))

total groups before partinioning : 2
=== Starting New Pass ===
Iteration 2, group size: 4
Transition dict for group 2: {(0, 2): {{
  "isTerminatingState": false,
  "0": "S6",
  "1": "S5"
}}, (2, 2): {{
  "isTerminatingState": false,
  "0": "S4",
  "1": "S5"
}, {
  "isTerminatingState": false,
  "0": "S4",
  "1": "S5"
}}, (1, 2): {{
  "isTerminatingState": false,
  "1": "S2",
  "0": "S3"
}}}
Transition key: (0, 2), states: {{
  "isTerminatingState": false,
  "0": "S6",
  "1": "S5"
}}
Transition key: (2, 2), states: {{
  "isTerminatingState": false,
  "0": "S4",
  "1": "S5"
}, {
  "isTerminatingState": false,
  "0": "S4",
  "1": "S5"
}}
Transition key: (1, 2), states: {{
  "isTerminatingState": false,
  "1": "S2",
  "0": "S3"
}}
New groups size: 5
=== Starting New Pass ===
Iteration 3, group size: 2
Transition dict for group 3: {(2, 3): {{
  "isTerminatingState": false,
  "0": "S4",
  "1": "S5"
}, {
  "isTerminatingState": false,
  "0": "S4",
  "1": "S5"
}}}
Transition key: (2, 3), state

[{{
    "isTerminatingState": true,
    "0": "S6",
    "1": "S5"
  }},
 {{
    "isTerminatingState": true
  }},
 {{
    "isTerminatingState": false,
    "0": "S6",
    "1": "S5"
  }},
 {{
    "isTerminatingState": false,
    "0": "S4",
    "1": "S5"
  },
  {
    "isTerminatingState": false,
    "0": "S4",
    "1": "S5"
  }},
 {{
    "isTerminatingState": false,
    "1": "S2",
    "0": "S3"
  }}]

In [ ]:
''' 
DFA Minimization  with explanation
'''
def minimizeDFA(dfa : DFA) -> DFA: 
    print(dfa)
    #^ PASS 1 
    # transition : Transition
    # Step 1 : split groups into accepting and non accepting states
    accepting_states : Set[DFAState] = set()
    none_accepting_states : Set[DFAState] = set()
    for state in dfa.states : 
        if state.isTerminatingState : 
            accepting_states.add(state)
        else :
            none_accepting_states.add(state)
    
    groups : List[Set[DFAState]] = [accepting_states ,  none_accepting_states] 
    #This function takes the list of all set of groups and returns
    #the group number , : g1,g2,g3 and so on to be used in comparing and splitting
    def get_group_index(state: DFAState, groups: List[Set[DFAState]]) -> int:
        for i, group in enumerate(groups):
            if state in group:
                return i
    '''
    example output for this function : 
    transition_dict = {
    el input el wlany wadany group 1
    el input el tany wdany group 2 

    (1, 2): {S1, S2},  # States S1 and S2 have identical transitions
    (0, 0): {S3, S4}   # States S3 and S4 have identical transitions
    }
    each tuple corresponds to a transition and we are sure thet have same transitions
    from pass 1 
    '''
    def create_transition_dict(subgroup: Set[DFAState],  groups: List[Set[DFAState]]) -> Dict[Tuple[int, ...], Set[DFAState]]:
        transition_dict: Dict[Tuple[int, ...], Set[DFAState]] = {}
        # Get List of inputs in which all these states have by selecting any one of them
        # I chose the first one for simplicity , hya msh far2a 
        #klhom 3ndhom nafs en inputs aslan men PASS 1 
        #assume the inputs for this subgroup is (a,b,c)
        #then every state in this subgroup goes to some destination 
        #on inputs a,b,c
        input_symbols = list(subgroup)[0].get_transitions_inputs() 
        # loop on every state and see The destination of each symbol , let us take the first state for  example S0 : 
            #  S0.transitions[a] -> goes to group 1
            #  S0.transitions[b] -> goes to group 5
            #  S0.transitions[c] -> goes to group 6 
            #  now we create a tuple key of (1,5,6) where each index xorresponsd to input symbol and the value is the group destination
            # search in the dict if theres a key of the same values and order  : (1,5,6)
                #if the key doesnt exist then create it with value S0 
                    # dict . add (1,5,6) :{S0}
                #else if the ke exist before , for example we have this : (1,5,6) :{S1} 
                #then this means that state S1 on input a went to g1 and in input b went to g5 and on input c went to g6
                #so S1 and S0 are acting exactly the same 
                    # Add S0 to the entry found : (1,5,6) :{S0,S1}
            #AND SO ON , AT THE END ALL STATES THAT ACTS THE SAME ARE TOGETHER IN ONE SET 
        for state in subgroup :
            transition_key_elements = []
            for input_symbol in input_symbols:
                destination_state = state.transitions[input_symbol]
                
                # Get the index of the group to which the destination state belongs
                group_index = get_group_index(destination_state, groups)
                transition_key_elements.append(group_index)
                transition_key = tuple(transition_key_elements)

                # Group states by their transition pattern
                if transition_key in transition_dict:
                    transition_dict[transition_key].add(state)
                else:
                    transition_dict[transition_key] = {state}
        return transition_dict


    for group in groups : 
        transitions_dict: Dict[Tuple[str, ...], Set[str]] = {}
        ''' 
        key             value 
        (a,b,c)         S0,S1,S2
        (c,d,x)         S4,S9 

        '''
        # Step 2 : for every group , split it into subgroups based on their inputs  
        for state in group:
            transitions = state.get_transitions_inputs()
            if transitions in transitions_dict:
                transitions_dict[transitions].add(state)
            else:
                transitions_dict[transitions] = [state] 
           
        # Step 3 : delete the group and add the subgroups to groups list 
         
        groups.remove(group)
        for value in transitions_dict.values() :
            groups.append(value)  
    print ("groups after pass1 :\n",groups)        
    #^ PASS TWO : 
    # Create a dictionary for each input in subgroup : 
    # now we are sure that every subgroup have same inputs
    # Main minimization loop
   
        # makontesh 3arf a3ml for loop 3l groups 3shan kont b8yr fl length bt3ha 
        # also ana msh 3arfa hwa2af emta fa el mfrod tkon while mesh for 
        # 3mlt new groups to update feha then compare if no changes happened a break , otherwise wna ba update
        # fe nafs el haga mkansh 3andy stop condition 
       #^ PASS TWO : 
    while True:
        new_groups: List[Set[DFAState]] = []
        print("=== Starting New Pass ===")
        for i, group in enumerate(groups):
            if(len(group) == 1):
                new_groups.append(group)
                continue 
            if(list(group)[0].get_transitions_inputs() == ()): # no transitions
                new_groups.append(group)
                continue
            print(f"Iteration {i}, group size: {len(group)}")
            transition_dict = create_transition_dict(group, groups)
            print(f"Transition dict for group {i}: {transition_dict}")
            
            # We will keep track of merged groups in new_groups
            for transition_key, states in transition_dict.items():
                print(f"Transition key: {transition_key}, states: {states}")
                new_groups.append(states)

        print(f"New groups size: {len(new_groups)}")
        
        if new_groups == groups:
            print("Groups have stabilized, exiting.")
            break

        groups = new_groups

    return groups

print (minimizeDFA(dfa))

        


    
    






{
  "startingState": "S1",
  "S1": {
    "isTerminatingState": false,
    "1": "S2",
    "0": "S3"
  },
  "S2": {
    "isTerminatingState": false,
    "1": "S4",
    "0": "S5"
  },
  "S3": {
    "isTerminatingState": true
  },
  "S4": {
    "isTerminatingState": false,
    "1": "S4",
    "0": "S5"
  },
  "S5": {
    "isTerminatingState": false,
    "1": "S4",
    "0": "S6"
  },
  "S6": {
    "isTerminatingState": true,
    "1": "S4",
    "0": "S6"
  }
}
groups after pass1 :
 [{{
  "isTerminatingState": false,
  "1": "S4",
  "0": "S5"
}, {
  "isTerminatingState": false,
  "1": "S4",
  "0": "S5"
}, {
  "isTerminatingState": false,
  "1": "S2",
  "0": "S3"
}, {
  "isTerminatingState": false,
  "1": "S4",
  "0": "S6"
}}, [{
  "isTerminatingState": true,
  "1": "S4",
  "0": "S6"
}], [{
  "isTerminatingState": true
}]]


KeyboardInterrupt: 